In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName("Movie").getOrCreate()

In [18]:
#arch = "D\\ ..... agrega la ruta"
arch = "movie_data_tmbd.csv"

In [19]:
df = spark.read.format("csv").option("inferSchema", False).option("header", True).option("sep", "|").load(arch)

In [20]:
df.printSchema()

root
 |-- adult: string (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string 

In [21]:
from pyspark.sql.types import *

In [22]:
float_vars=["popularity"]
date_vars=["release_date"]

In [23]:
for c in float_vars:
    df = df.withColumn(c, df[c].cast(FloatType()))
    
for c in date_vars:
    df = df.withColumn(c, df[c].cast(DateType()))

In [35]:
# Para calcular los valores faltantes en una columan o en
# una columan o en varias
# utilizando funciones de Spark
from pyspark.sql.functions import *

In [40]:
# 1) Identificar los títulos que están repetidos entre los años 2010 al 2020 y contar el número de repeticiones por cada título
df.agg(countDistinct(col("title")).count.filtrer((year("release_date") >= 2010)&(year("release_date") <= 2020))).show(10, False)

TypeError: 'Column' object is not callable

In [41]:
##
df.groupBy(df['title']).count.filtrer((year("release_date") >= 2010)&(year("release_date") <= 2020)).show(10, False)

AttributeError: 'function' object has no attribute 'filtrer'

In [38]:
###
df.select("title").distinct().count.filter((year("release_date") >= 2010)&(year("release_date") <= 2020)).show(10, False)

AttributeError: 'function' object has no attribute 'filter'

In [42]:
# 2) Desplegar todos los títulos que contengan la palabra "Harry"
df.filter(df.title.like('%Harry%')).show(10,False)

+-----+--------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+------+---------------------------------------------------------------------------------------------+---------------------------------------------------------------------+------+---------+-----------------+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [33]:
# 3) Agregar una nueva columna binaria que sirva como indicador si el idioma original es el inglés
df.withColumn("English", (decode(col("title"), "US-ASCII")) ).show(10, False)

+-----+--------------------------------+---------------------+------+-----------------------------------------------------------------------------------------------------------------------------+--------+-----+---------+-----------------+------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [39]:
# 4) Desplegar (tabla) el promedio de popularidad por año.
df.groupBy(year("release_date")).agg(mean("popularity")).show(10, False)

+------------------+------------------+
|year(release_date)|avg(popularity)   |
+------------------+------------------+
|1959              |2.6196672424648475|
|1990              |4.057833824433437 |
|1975              |2.8473358422443082|
|1977              |2.8324450663386975|
|1924              |2.2806603796077223|
|2003              |4.563239492305484 |
|2007              |4.556657047444546 |
|2018              |5.3192447457673415|
|1974              |2.7387080675983633|
|2015              |4.73031214037638  |
+------------------+------------------+
only showing top 10 rows

